In [ ]:
# Import packages
import os

import pandas as pd
import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from functools import reduce

from dataclasses import dataclass
from dataclasses import InitVar
from typing import List, Optional, Dict

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from statsmodels.api import formula as smf

import warnings

## 1. Bioaccumulation model functions

In [ ]:
# 1. Dataclasses for input parameters and submodels

@dataclass  
class Settings():
    """ Switches that control the use of alternative versions of variable parameterizations for 
    different model applications or model development/testing.
    
    Parameters with default settings across lab and field studies are given default values
    Parameters that vary between lab and field model versions are not given default values"""
    
    # all fish models
    chooseEd: str  # Martin or Goeritz
    chooseDiet: str     # default, zero, forced
    chooseStudyType: str = 'default' # default, Martin BCF, or Martin BMF
    chooseRenal: str = 'off' # 'on' (with renal elimination) or 'off' (without renal elimination)
    chooseDmw: str = 'Droge'
    chooseEw: str = 'empirical'
    
    # food web models
    chooseKoc: str = 'Koc_Munoz' 
    # placeholder for lab-controlled studies with no sediment; this parameter will have no effect on modeled concentration
    
        
@dataclass
class Environment():
    """ Dataclass for model parameters and input parameters that vary by ecosystem"""
    
    C_OX: float # dissolved oxygen concentration (mg/L)
    T: float  # temperature (C)
    C_SS: float # concentration of suspended solids (kg/L)
    OCS: float # organic carbon content of the sediment (kg/L)
    pH: float # pH
    pHi: float = 7.4 # default for internal pH as suggested by Armitage et  al. 2013
        
    def __post_init__(self):
        self.pHg = self.pH - 1       
           

@dataclass 
class Chemical():
    """ Dataclass that holds parameters unique to each PFAS chemical. Parameters include physiochemial properties, pH-dependent properties (varies with environment), 
    and empirically-derived parameter values (sometimes varies with organism characteristics/study; this choice is made
    during the instantiation of this class)
    
    Functions include calculation of chemical-specific parameters, which may depend on environmental parameters (class Environment)"""
    
    Log_Kow: float 
    Log_Kpw: float
    Log_Dmw: float
    pKa: float
    Ed_exp: float
    Ew_exp: float
    Log_Koc: float
    chemID: int
    env: InitVar[Environment]
    
    
    def __post_init__(self, env: Environment):
        self.K_OW = 10**self.Log_Kow
        self.K_PW = 10**self.Log_Kpw
        self.D_MW_exp = 10**self.Log_Dmw
        self.K_OC = 10**self.Log_Koc
        
        self.D_OW = self.getD_OW(env)
        
    def getD_OW (self, env: Environment):
        """ Octanol-water distribution coefficient (Armitage et al. 2013)"""        
        Xn = 1 / (1 + 10**(env.pHi-self.pKa)) # neutral fraction of the compound
        Xi = 1 - Xn # ionic fraction of the compound

        logK_OW_i = np.log10(self.K_OW) - 3.1# based on Armitage et al 2013, see SI Table S7
        K_OW_i = 10 ** logK_OW_i 

        D_OW = Xn * self.K_OW + Xi * K_OW_i        
        
        assert np.log10(D_OW) > 0, 'error in D_OW calculation (negative value)' 
        
        return D_OW
        
    def getD_MW (self, env: Environment, settings: Settings): 
        if settings.chooseDmw in ('Droge') :
            D_MW = self.D_MW_exp
        elif settings.chooseDmw == 'calculated': 
            Xn = 1 / (1 + 10**(env.pHi-self.pKa)) # neutral fraction of the compound
            Xi = 1 - Xn # ionic fraction of the compound

            a = 1.01 # Endo et al, as cited in Armitage et al 2013, Table S2
            b = 0.12 # Endo et al, as cited in Armitage et al 2013
            logK_MW_n = (a * np.log10(self.K_OW)) + b
            K_MW_n = 10**logK_MW_n

            logK_MW_i = np.log10(K_MW_n) - 2 # based on Armitage et al 2013, see SI Table S7
            K_MW_i = 10**logK_MW_i

            D_MW = Xn * K_MW_n + Xi * K_MW_i
        else:
            assert 'error in D_MW choice'

        assert np.log10(D_MW) > 0, "error in D_MW calculation (negative value)"
        
        return D_MW
    
    
    def getE_W(self, env: Environment, settings: Settings):  
        """ Fish gill uptake efficiency (%)
        Used to calculate k1 clearance rate constant for fish, invertebrates, and zooplankton  
        """
        
        if 'beta' in settings.chooseEw:
            pHg = env.pHi - 1
            Ew_mu = 1 + 10**(pHg - self.pKa)
            Ew_beta = float(settings.chooseEw.replace('beta',''))
            E_W = (1 / (1.85 + (Ew_mu * 155/self.K_OW))) + Ew_beta 
        elif settings.chooseEw in ('empirical'):
            E_W = self.Ew_exp
        else:
            assert 'error in Ew selection'
            
        return E_W
            
   
    def getE_D(self, settings: Settings): 
        """ Dietary chemical transfer efficiency (%) 
        Used to calculate kd uptake rate constant. 
        Empirical values for PFAAs differ for juvenile and adult fish
        """
        
        if settings.chooseEd == 'calculated': 
        # note that Aed and Bed are based on empirical relationships that are highly variable; these are for fish 
            A_ED = 3 *  10**(-7) # front Arnot & Gobas 2004
            B_ED = 2
            E_D = (A_ED * self.K_OW  + B_ED)**(-1)
        elif settings.chooseEd in ('Martin','Goeritz'):
            E_D = self.Ed_exp
        else:
            assert 'error in Ed selection'
            
        return E_D
    
    
@dataclass
class Organism():
    """ Dataclass that holds parameters unique to each organism, or the calculation of parameters that may vary
    depending on organism type (i.e. sigma, beta)
    
    Functions include calculations of (1) diet composition; (2) bioenergetic rates; (3) partitioning coefficients;
    and (4) chemical uptake and elimination rate constants"""

    W_B: float
    m_O: float
    GRF: float
    
    nu_NB: float
    nu_LB: float
    nu_PB: float
    nu_OB: float
    nu_WB: float
    
    nu_ND: float
    nu_LD: float
    nu_PD: float
    nu_OD: float
    nu_WD: float
    
    epsilon_N: float
    epsilon_L: float
    epsilon_P: float
    epsilon_O: float
    epsilon_W: float

    switchk_1: int
    switchG_D: int
    switchk_R: int
        
    A: float
    B: float
    sigma: float 
    
    # Diet functions
    def _getnu_NG(self):
        nu_NG = (1 - self.epsilon_N) * self.nu_ND / ((1- self.epsilon_N) * self.nu_ND + (1 - self.epsilon_L) * self.nu_LD + 
                                    (1 - self.epsilon_P)* self.nu_PD + (1 - self.epsilon_O) * self.nu_OD + (1 - self.epsilon_W) * self.nu_WD)
        return nu_NG

    def _getnu_LG(self):
        nu_LG = (1 - self.epsilon_L) * self.nu_LD / ((1- self.epsilon_N) * self.nu_ND + (1 - self.epsilon_L) * self.nu_LD + 
                                           (1 - self.epsilon_P)* self.nu_PD + (1 - self.epsilon_O) * self.nu_OD + (1 - self.epsilon_W) * self.nu_WD)
        return nu_LG

    def _getnu_PG(self):
        nu_PG = (1 - self.epsilon_P) * self.nu_PD / ((1- self.epsilon_N) * self.nu_ND + (1 - self.epsilon_L) * self.nu_LD + 
                                           (1 - self.epsilon_P)* self.nu_PD + (1 - self.epsilon_O) * self.nu_OD + (1 - self.epsilon_W) * self.nu_WD)
        return nu_PG

    def _getnu_OG(self):
        nu_OG = (1 - self.epsilon_O) * self.nu_OD / ((1- self.epsilon_N) * self.nu_ND + (1 - self.epsilon_L) * self.nu_LD + 
                                           (1 - self.epsilon_P)* self.nu_PD + (1 - self.epsilon_O) * self.nu_OD + (1 - self.epsilon_W) * self.nu_WD)
        return nu_OG

    def _getnu_WG(self):
        nu_WG = (1 - self.epsilon_W) * self.nu_WD / ((1- self.epsilon_N) * self.nu_ND + (1 - self.epsilon_L) * self.nu_LD + 
                                           (1 - self.epsilon_P)* self.nu_PD + (1 - self.epsilon_O) * self.nu_OD + (1 - self.epsilon_W) * self.nu_WD)
        return nu_WG
    

    # Organism functions
    
    def _getG_V(self, env: Environment, settings: Settings):
        "G_V is the ventilation rate. C_OX is the dissolved oxygen concentration (mg/L)"
        
        G_V = 1400 * (self.W_B**.65) / env.C_OX 
        
        return G_V

    def _getG_D(self, env: Environment, settings: Settings): 
        """ G_D is the feeding rate
        G_Dswitch determines which formula to use based on the type of organism and data availability
        1 should be used to estimate G_D for coldwater fish species, and in some cases, zooplankton
        2 should be used to estimate G_D for filter-feeding species
        Use empirical data where available, though be careful about the energetic content of food and associated growth rates"""

        if self.switchG_D == 0:
            return float('NaN') 
        elif self.switchG_D == 1: 
        # Note that several studies have observed decreasing concentrations with increasing fish length w/in the same species
        # This could potentially be due to growth dilution, onotogenetic shifts, or other processes, not necessarily feeding rate
            if settings.chooseStudyType in ('Martin BCF', 'Martin BMF'):
                G_D = self.W_B * 0.015 # 1.5% bw feeding rate
            else:
                G_D = 0.022 * self.W_B**0.85 * np.exp(0.06*env.T) 
            return G_D
        elif self.switchG_D == 2:        
            #G_V is the gill ventilation rate
            #C_SS is the concentration of suspended solids
            #sigma is the scavenging efficiency of particles
            G_V = self._getG_V(env, settings)
            G_D = G_V * env.C_SS * self.sigma
            return G_D    
        else:
            return "error in getG_D: wrong input for switchG_D argument"
    
    def _getG_F(self, env: Environment, settings: Settings): 
    # G_F is the fecal egestion rate
    # epsilon_L, epsilon_N, and epsilon_W are the dietary assimilation efficiencies of lipid, NLOM and water, respectively.
    # nu_LD, nu_ND, and nu_WD are the overall lipid, NLOM and water contents of the diet, respectively.
        G_D = self._getG_D(env, settings)
        G_F = ((1 - self.epsilon_N) * self.nu_ND + (1 - self.epsilon_L) * self.nu_LD + (1 - self.epsilon_P) * self.nu_PD + (1 - self.epsilon_O) * self.nu_OD + (1 - self.epsilon_W) * self.nu_WD) * G_D

        return G_F
    
    
    # Partitioning coefficients
    
    def getD_BW (self, chem: Chemical, env: Environment, settings: Settings):
        D_OW = chem.getD_OW(env)
        D_MW = chem.getD_MW(env, settings)

        if self.switchk_1 == 0:
            Beta = 0.35  
        else:
            Beta = 0.05 
        
        D_BW = self.nu_NB * D_OW + self.nu_LB * D_MW + self.nu_PB * chem.K_PW + self.nu_OB * D_OW * Beta + self.nu_WB
        
        return D_BW
    

    def _getK_GB(self, chem: Chemical, env: Environment, settings: Settings):
        """ K_GB is the partition coefficient of the chemical between the contents of GIT and the oragnism.
        nu_NG, nu_LG, nu_PG and nu_WG are the neutral lipid, phospholipid, protein and water contents,respectively, in the gut."""

        nu_NG = self._getnu_NG() 
        nu_LG = self._getnu_LG()
        nu_PG = self._getnu_PG()
        nu_OG = self._getnu_OG()
        nu_WG = self._getnu_WG()

        D_OW = chem.getD_OW(env)
        D_MW = chem.getD_MW(env, settings)
        D_BW = self.getD_BW(chem, env, settings)
        
        D_GW = nu_NG * D_OW + nu_LG * D_MW + nu_PG * chem.K_PW + nu_OG * D_OW * 0.05 + nu_WG

        K_GB = D_GW / D_BW
        
        return K_GB


    def _getK_BU (self, chem: Chemical, env: Environment, settings: Settings):
        # K_BU is the partition coefficient between the organism and urine C_B/C_U

        #K_BW is the organism water partition coefficient
        #nu_LB is the lipid fraction (kg lipid/kg organisms ww)
        #nu_NB is the NLOM fraction (kg NLOM/kg organism ww)
        #nu_WB is the water content (kg water/kg organism ww) of the organism
        #Beta is a proportionality constant expressing the sorption capacity of NLOM to that of octanol
        #.035 is a reasonable estimate of Beta

        LipidDensity=0.9 # kg/L
        PLipidDensity=0.9 # kg/L (Armitage et al. 2013)
        ProteinDensity= 1.35 # g/cm3 = kg/L (Fischer et al. 2009; Allendorf et al. 2020) 
        NLOMDensity=1 # kg/L

        D_OW = chem.getD_OW(env)
        D_MW = chem.getD_MW (env, settings)
        
        K_BU = (self.nu_NB * D_OW / LipidDensity) + (self.nu_LB * D_MW / PLipidDensity) + (self.nu_PB * chem.K_PW / ProteinDensity) + (self.nu_OB * D_OW * 0.05 / NLOMDensity) + self.nu_WB
        
        return K_BU
    
    
    # Rate constant functions 
    
    def getk_1(self, chem: Chemical, env: Environment, settings: Settings):
        """ k_1 is the clearance rate constant (L/kg*d) for chemical uptake via respiratory area (i.e., gills and skin) """
    
    
        if self.switchk_1 == 0:    
            #A and B are constants describing the reistance to chemical uptake 
            #through aqueous (A) and organic phases (B) of the algae, phytoplankton, or macrophyte.
            #A default value = 6x10^-5 for PCBs in Arnot & Gobas 2004; 8.2 x 10^-3 in Gobas & MacKay 1987
            #B default value = 5.5 for PCBs in Arnot & Gobas 2004; 0.68 in Gobas & MacKay 1987
            # difference between these two options is minimal

            #k_1 for algae, phytoplankton, and aquatic macrophytes

            k_1 = 1/(self.A + (self.B / chem.D_MW_exp))
            
            return k_1

        elif self.switchk_1 == 1:

            #G_V is ventilation rate (should be able to find and not use equation)
            #G_V can be approximated by G_V = 1400 * (W_B^.65)/C_OX where C_OX = (-.24*T + 14.04) * S
            #W_B is the wet weight of the organism (kg)
            #E_W is the gill chemical uptake efficiency

            G_V = self._getG_V(env, settings)
            E_W = chem.getE_W(env, settings)

            #k_1 for fish, invertebrates, and zooplankton
            k_1 = E_W * G_V / self.W_B

            return k_1
        
            
    def getk_D(self, chem: Chemical, env: Environment, settings: Settings): 
        """ Dietary uptake clearance constant (kg food/kg organism per day)
        The rate at which chemicals are absorbed from the diet via the GIT"""
        #E_D is the dietary chemical transfer efficiency
        #G_D is the feeding rate (kg food/day) - empirical data often available
        #W_B is the weight of the organism (kg organism) 
        E_D = chem.getE_D(settings)
        G_D = self._getG_D(env, settings)

        k_D = E_D * G_D / self.W_B

        return k_D
    
    def getk_E(self, chem: Chemical, env: Environment, settings: Settings):
        """ Fecal elimination rate constant: the rate at which chemicals are eliminated by the egestion of fecal matter (1/d) """
        #G_F is the fecal egestion rate (kg-feces/kg-organism/day)
        #E_D is the dietary chemical transfer efficiency
        #K_GB is the partition coefficient of the chemical between the GIT and the organism.
        #W_B is the weight of the oyrganism
        #epsilon_L, epsilon_N, and epsilon_W are the dietary assimilation efficiencies of lipid, NLOM, and water, respectively
        #nu_LD, nu_ND, and nu_WD are the overall lipid, NLOM, and water contents of the diet

        G_F = self._getG_F(env, settings)
        E_D = chem.getE_D(settings)
        K_GB = self._getK_GB(chem, env, settings)

        k_E = G_F * E_D * K_GB / self.W_B

        return k_E
    
    def getk_G(self, settings: Settings):
        """ Growth rate constant. 
        W_B is the weight of the organism at time t, 
        GRF is the growth rate factor that differs by organism and temperature"""
        if settings.chooseStudyType in ('default','reduced feeding'):
            if self.switchk_1 ==0: # algae, phytoplankton, macrophytes
                k_G = self.GRF # GRF input for phytoplankton is simply the growth rate constant 
                return k_G
            elif self.switchk_1 == 1: # fish, invertebrates
                k_G = self.GRF* self.W_B**(-0.2) 
                return k_G
            else:
                return 'error in getk_G: wrong input for switchk_1 argument'
        elif settings.chooseStudyType in ('Martin BCF', 'Martin BMF', 'Martin BMF rev'):
            return 0 
        else:
            return 'error in getk_G: wrong input for chooseStudyType argument'
    

@dataclass
class ChemData():
    """ Dataclass that holds chemical data that is unique to each model system. This includes both model parameters and input variables"""
    
    C_WTO: float
    C_s: float
    Phi: float # should be < 1; default is 1 for the lab studies if this is not used
        
    def getC_WDP(self, chem: Chemical, env: Environment, settings: Settings):
        """ Freely dissolved chemical concentration in the sediment associated pore (or interstitial) water (g/L)"""
    #C_WDP = C_SOC / K_OC where C_SOC is the chemical concentration in sediment normalized for OC content (g/kg-OC)
            
    #Cs is the Concentration in sediment
    #OCS is the organic carbon content of the sediment
    #Koc is the organic carbon partitioning coefficient; Koc = Kd / OCS 
    
        C_WDP = (self.C_s / env.OCS) / chem.K_OC

        return C_WDP
        
        


In [ ]:
# 2. Steady State Equation for calculating concentration in a single organism

def getSSC_B(settings: Settings, chemdata: ChemData, C_D: float, P: float, Pd: pd.DataFrame, \
             env: Environment, chem: Chemical, org: Organism, kRTable: Optional[pd.DataFrame]=None):
        
        ##############################################
        ## Calculate intermediate model inputs & rates
        ##############################################
        
        C_WDP = chemdata.getC_WDP(chem, env, settings) 

        k_1 = org.getk_1(chem, env, settings) #k_1 is the clearance rate constant (L/kg*d) for chemical uptake via respiratory area
        k_2 = k_1 / org.getD_BW(chem, env, settings)
        
        if org.switchk_1 == 0: # Phytoplankton
            k_D = 0
            k_E = 0
            k_G = org.GRF
        elif org.switchk_1 == 1: # Zooplankton, Aquatic Invertebrates, Fish
            k_D = org.getk_D(chem, env, settings) 
            #k_E is the rate constant (1/d) for chemical elimination via exretion into egested feces
            k_E = org.getk_E(chem, env, settings) 
            #k_G is the growth rate constant
            k_G = org.getk_G(settings) 
        else:
            return 'error in k1 switch selection for dietary & growth uptake & elimination pathways'
    
        k_M = 0
            
        # Renal elimination rate
        if settings.chooseRenal == 'on':
            if org.switchk_R == 1:
                # df_krkbRatio
                k_R_est = kRTable.loc[chem.chemID,'kr/kb'] * k_2
            elif org.switchk_R == 0:
                k_R_est = 0
            else:
                assert 'error in kR switch'
        elif settings.chooseRenal == 'off':
            k_R_est = 0
        else:
            return 'error in renal elimination settings'
        
        ##################################
        ## Calculate tissue concentration 
        ##################################
        C_B = ((k_1 * (org.m_O * chemdata.Phi * chemdata.C_WTO + (1-org.m_O) * C_WDP) + \
                k_D * (sum(P * C_D) + (Pd * chemdata.C_s))) / (k_2 + k_E + k_M + k_G+ k_R_est))
        
       
        #################################
        ## Create table of output values 
        ################################
        D_OW = chem.getD_OW(env)
        D_MW = chem.getD_MW(env, settings)
        D_BW = org.getD_BW(chem, env, settings)
                       
        G_F = org._getG_F(env, settings)
        #G_F = 0
        G_D = org._getG_D(env, settings)
        #G_D = 1
        G_V = org._getG_V(env, settings)
        E_D = chem.getE_D(settings)
        K_GB = org._getK_GB(chem, env, settings)
        E_W = chem.getE_W(env, settings)

        Diet = sum(P * C_D) + (Pd * chemdata.C_s)
        Water = (org.m_O * chemdata.Phi * chemdata.C_WTO + (1-org.m_O) * C_WDP)
        FeedRate = G_D / org.W_B
        Gill_uptake = k_1 * (org.m_O * chemdata.Phi * chemdata.C_WTO + (1-org.m_O) * C_WDP) # L/kg*d * ng/mL (or g/L) = g chemical/kg fish/day
        Dietary_uptake = k_D * (sum(P * C_D) + (Pd * chemdata.C_s)) # kg food/kg org * ng/g (or g/kg food) = g chemical/kg fish/day
        Uptake = Gill_uptake + Dietary_uptake
        Gill_uppct = Gill_uptake / Uptake
        Diet_uppct = Dietary_uptake / Uptake
        TotalElim_rate = (k_2 + k_E + k_M + k_G)
        
        #FoodP = sum(P)
        #FoodD = sum(C_D)
        #Sediment = Pd * C_s
        
        NL_pct = (org.nu_NB * D_OW) / D_BW
        PL_pct = (org.nu_LB * D_MW) / D_BW
        Protein_pct = (org.nu_PB * chem.K_PW) / D_BW
        NLOM_pct = (org.nu_OB * D_OW * 0.05) / D_BW
        water_pct = (org.nu_WB) / D_BW   
        
        kr_pct = k_R_est / (k_2 + k_E + k_G + k_R_est)
        
        
        
        Output_Data = np.array([C_B, org.m_O, chemdata.Phi,chemdata.C_WTO, C_WDP, Water, chemdata.C_s, FeedRate, Diet, Gill_uptake, Dietary_uptake, Gill_uppct, Diet_uppct, G_V, G_D, G_F, org.W_B, E_W, E_D, 
                                k_1, k_2, k_D, k_E, k_G, k_R_est, TotalElim_rate, kr_pct, chem.pKa, np.log10(D_BW), chem.Log_Kow, np.log10(D_MW), np.log10(D_OW), chem.Log_Kpw, 
                                D_BW, D_MW, D_OW, chem.K_PW, env.pHi, env.pHg, K_GB, org.nu_NB, org.nu_LB, org.nu_PB, org.nu_OB, org.nu_WB, 
                                org.epsilon_N, org.epsilon_L, org.epsilon_P, org.epsilon_O, chem.Log_Koc,chemdata.Phi])

        return Output_Data


In [ ]:
# 3. Food Web Model

def BioaccumulationModel(PFAA: str, settings_dict: Dict[str,pd.DataFrame], numSpecies: int, oceanData: pd.DataFrame, \
                         chemicalData: pd.DataFrame, chemicalParams: pd.DataFrame, organismData: pd.DataFrame, \
                         foodWebData: pd.DataFrame, dietData:Optional[pd.DataFrame]=None, kRTable:Optional[pd.DataFrame]=None): 
    
    #######################
    # Set up model settings
    #######################
    settings = Settings(**settings_dict)
    chemicalData = chemicalData[PFAA]
    chemicalParams = chemicalParams[PFAA]
    if dietData is not None:
        dietData = dietData[PFAA]
    
    ####################
    # Read in parameters
    ####################
    ## Ocean Parameters
    C_OX = oceanData.loc['C_OX'] #Dissolved Oxygen Concentration (mg/L)
    T = oceanData.loc['T'] #Temperature (C)
    C_SS = oceanData.loc['C_SS'] #Concentration of suspended solids (kg/L)
    OCS = oceanData.loc['OCS'] #Organic carbon content of the sediment (fraction)
    pH = oceanData.loc['pH'] # ocean pH 
    pHi = 7.4 # default for internal pH as suggested by Armitage et  al. 2013.
    pHg = pH - 1 

    ## Chemical Parameters
    Log_Kow = chemicalParams.loc['Log_Kow_COSMOtherm']
    Log_Kpw = np.log10(1.36 * 10**chemicalParams.loc['log_Kpw_Allendorf'])
    Log_Dmw = chemicalParams.loc['log_Dmw_droge'] # Dmw measured by Droge 2019
    pKa = chemicalParams.loc['pKa_Armitage'] 
    Log_Koc = chemicalParams.loc['log_Koc_Munoz'] # choose values relevant for model system
    E_W_exp = chemicalParams.loc['E_W_Martin10'] # calculated at Cox = 10 mg/L. Estimated based on T=12, DO sat approximately 92%
    
    
    # Ed
    if settings.chooseEd == 'Martin':
        E_D_exp = chemicalParams.loc['E_D_Martin2003']
    elif settings.chooseEd == 'Goeritz':
        E_D_exp = chemicalParams.loc['E_D_Goeritz']
    else:
        assert 'error in Ed selection'
        
    # chemID
    chemID = chemicalParams.loc['chemID']
    
    
    ## Organism Parameters
    W_B = organismData.loc['W_B',:]  #Final Weight of the organism (kg)
    m_O = organismData.loc['m_O',:] #Fraction of the respiratory ventilation that involves overlying water
    
    nu_NB = organismData.loc['nu_NB',:]  #Neutral lipid fraction
    nu_LB = organismData.loc['nu_LB',:]  #Phosphoipid fraction
    nu_PB = organismData.loc['nu_PB',:]  #Protein fraction
    nu_OB = organismData.loc['nu_OB',:]  #NLOM (or "other") fraction
    nu_WB = organismData.loc['nu_WB',:]  #Water content
    
    epsilon_N = organismData.loc['epsilon_N',:]  #Dietary assimilation efficiency of neutral lipid contents of diet (fraction)
    epsilon_L = organismData.loc['epsilon_L',:]  #Dietary assimilation efficiency of phospholipid contents of diet ((fraction)
    epsilon_P = organismData.loc['epsilon_P',:]  #Dietary assimilation efficiency of protein contents of diet (fraction)
    epsilon_O = organismData.loc['epsilon_O',:]  #Dietary assimilation efficiency of the NLOM content of diet (fraction)
    epsilon_W = organismData.loc['epsilon_W',:]  #Dietary assimilation efficiency of water contents of diet (fraction)
    
    A = organismData.loc['A',:]  #Aqueous phase resistance constant for k1 calculation
    B = organismData.loc['B',:]  #Organic phase resistance constant for k1 calculation
    GRF = organismData.loc['GRF',:] #growth weight factor
    
    switchk_1 = organismData.loc['switchk_1',:] #dependent on organism type
    switchG_D = organismData.loc['switchG_D',:]  #dependent on organism type
    switchk_R = organismData.loc['switchk_R',:] # dependent on organism type
    
    sigma =  1 # sigma is the scavenging efficiency of particles
    Beta =  0.035 # Proportionality constant representing the sorption capacity of NLOM to that of octanol.
    # Beta is specifically relevant to calculating partitioning in phytoplankton, where NLOM is replaced with NLOC    
    
    
    ####################################
    # Read in system-specific parameters
    ####################################
    ## System-Specific Chemical Data
    C_WTO = chemicalData.loc['C_WTO'] #Total chemical concentration in the water column above the sediments (g/L)
    C_s = chemicalData.loc['C_s'] #.0358; %C_s is the Concentration in sediment
    Phi = chemicalData.loc['Phi_exp'] # Field-measured Phi reported for a given system
    
    # Food Web Data
    Pd=foodWebData.iloc[:,0] #Fraction of the diet consisting of detritus
    P=foodWebData.iloc[:,1:numSpecies+1] #Fraction of the diet consisting of prey item i 
    
    ###############################
    # Calculate dietary composition
    ###############################
    
    nu_ND = np.zeros(numSpecies)
    nu_LD = np.zeros(numSpecies)
    nu_PD = np.zeros(numSpecies)
    nu_OD = np.zeros(numSpecies)
    nu_WD = np.zeros(numSpecies)
    
    for i in range(numSpecies):
        
        if settings.chooseStudyType == 'default':
            nu_ND[i] = sum((P.iloc[i,:] * nu_NB)) #Overall neutral lipid content of diet
            nu_LD[i] = sum((P.iloc[i,:] * nu_LB)) #Overall phospholipid content of the diet
            nu_PD[i] = sum((P.iloc[i,:] * nu_PB)) #Overall protein content of the diet
            nu_OD[i] = sum((P.iloc[i,:] * nu_OB)) #Overall NLOM content of the diet
            nu_WD[i] = sum((P.iloc[i,:] * nu_WB)) #Overall water content of diet
        elif settings.chooseStudyType in ('Martin BCF', 'Martin BMF','Martin BMF rev'):
            nu_ND[i] = 0.012
            nu_LD[i] = 0.003
            nu_PD[i] = 0
            nu_OD[i] = 0.15
            nu_WD[i] = 0.835
    
    
    ######################
    # Instantiate classes
    ######################
    env = Environment(C_OX, T, C_SS, OCS, pH)
    chem = Chemical(Log_Kow, Log_Kpw, Log_Dmw, pKa, E_D_exp, E_W_exp, Log_Koc, chemID, env)
    chemdata = ChemData(C_WTO, C_s, Phi)

    
    ############################
    # Run model and save results
    ############################
    
    Parameters = np.array(['C_B','Phi','mO','C_WTO','C_WDP','Water','C_s','FeedRate','Diet','Gill_uptake','Dietary_uptake','Gill_up%','Diet_up%','G_V','G_D','G_F','W_B','Ew','Ed',
                       'k1','k2', 'kd','ke','kg','kr_est','Total Elimination','kr_pct','pKa','logDbw','log Kow', 'log Dmw','log Dow','log Kpw',
                       'D_BW','D_MW','D_OW','K_PW','pHi','pHg','K_GB','nu_NB','nu_LB','nu_PB','nu_OB','nu_WB','epsilon_N','epsilon_L','epsilon_P', 'epsilon_O',
                       'Log_Koc','Phi'])
    
    
    Results = np.zeros((numSpecies,len(Parameters)))
    Results = pd.DataFrame(Results, columns=Parameters)
    Results.set_index(organismData.columns.values, inplace=True)
    
    if settings.chooseDiet == 'default':
        C_D = np.zeros(numSpecies)
        for i in range(0,numSpecies):
            orgi = Organism(W_B[i], m_O[i], GRF[i], nu_NB[i], nu_LB[i], nu_PB[i], nu_OB[i], nu_WB[i], 
                            nu_ND[i], nu_LD[i], nu_PD[i], nu_OD[i], nu_WD[i], epsilon_N[i], epsilon_L[i], epsilon_P[i], 
                            epsilon_O[i], epsilon_W[i], switchk_1[i], switchG_D[i], switchk_R[i],  A[i], B[i], sigma) 
            Results.iloc[i,:] = getSSC_B(settings, chemdata, C_D, P.iloc[i,:], Pd[i], env, chem, orgi,kRTable)
            C_D[i] = Results.iloc[i,0]
    elif settings.chooseDiet  == 'zero': 
        C_D = np.zeros(numSpecies)
        for i in range(0,numSpecies):
        #for i in range(0,1):
            orgi = Organism(W_B[i], m_O[i], GRF[i], nu_NB[i], nu_LB[i], nu_PB[i], nu_OB[i], nu_WB[i], 
                            nu_ND[i], nu_LD[i], nu_PD[i], nu_OD[i], nu_WD[i], epsilon_N[i], epsilon_L[i], epsilon_P[i], 
                            epsilon_O[i], epsilon_W[i], switchk_1[i], switchG_D[i], switchk_R[i], A[i], B[i], sigma) 
            Results.iloc[i,:] = getSSC_B(settings, chemdata, C_D, P.iloc[i,:], Pd[i], env, chem, orgi,kRTable)
            test = getSSC_B(settings, chemdata, C_D, P.iloc[i,:], Pd[i], env, chem, orgi)
            
    elif settings.chooseDiet in ('Martin BMF', 'Martin BMF rev'):
        C_D = [chemicalData.loc['C_D'], 0]
        for i in range(0,numSpecies):
            orgi = Organism(W_B[i], m_O[i], GRF[i], nu_NB[i], nu_LB[i], nu_PB[i], nu_OB[i], nu_WB[i], 
                            nu_ND[i], nu_LD[i], nu_PD[i], nu_OD[i], nu_WD[i], epsilon_N[i], epsilon_L[i], epsilon_P[i], 
                            epsilon_O[i], epsilon_W[i], switchk_1[i], switchG_D[i], switchk_R[i],  A[i], B[i], sigma) 
            Results.iloc[i,:] = getSSC_B(settings, chemdata, C_D, P.iloc[i,:], Pd[i], env, chem, orgi,kRTable)
    elif settings.chooseDiet == 'forced Munoz': # manual input
        # read in empirical diet data
        C_D = np.array(dietData)
        # insert modeled phytoplankton concentration, which is not empirically measured
        org0 = Organism(W_B[0], m_O[0], GRF[0], nu_NB[0], nu_LB[0], nu_PB[0], nu_OB[0], nu_WB[0], 
                        nu_ND[0], nu_LD[0], nu_PD[0], nu_OD[0], nu_WD[0], epsilon_N[0], epsilon_L[0], epsilon_P[0], 
                        epsilon_O[0], epsilon_W[0], switchk_1[0], switchG_D[0], switchk_R[0], A[0], B[0], sigma) 
        Results.iloc[0,:] = getSSC_B(settings, chemdata, C_D, P.iloc[i,:], Pd[i], env, chem, org0,kRTable)
        C_D[0] = Results.iloc[0,0]     
        for i in range(0,numSpecies):
            orgi = Organism(W_B[i], m_O[i], GRF[i], nu_NB[i], nu_LB[i], nu_PB[i], nu_OB[i], nu_WB[i], 
                            nu_ND[i], nu_LD[i], nu_PD[i], nu_OD[i], nu_WD[i], epsilon_N[i], epsilon_L[i], epsilon_P[i], 
                            epsilon_O[i], epsilon_W[i], switchk_1[i], switchG_D[i], switchk_R[i], A[i], B[i], sigma) 
            Results.iloc[i,:] = getSSC_B(settings, chemdata, C_D, P.iloc[i,:], Pd[i], env, chem, orgi,kRTable)
            
            
    # results post-processing
    Results['C_B'] = Results['C_B']*1000
    Results['logC_B'] = np.log10(Results['C_B'])
    Results['C_B_ngg'] = Results['C_B'] / 1000
    Results['logBAF'] = np.log10(Results['C_B_ngg'] / Results['C_WTO'])
    Results['BMF'] = Results['C_B_ngg'] / Results['Diet']

    return Results
    

    

## 2. Set up model inputs

In [ ]:
# Set up base model settings

# Gironde Estuary, observed diet without renal elmination
settings_GE_obsDiet = {
    'chooseStudyType': 'default',
    'chooseDiet': 'forced Munoz',
    'chooseEd': 'Goeritz',
    'chooseKoc': 'Koc_Munoz', 
    'chooseRenal': 'off'
}

# Gironde Estuary, modeled diet without renal elimination
settings_GE_modelDiet = settings_GE_obsDiet.copy()
settings_GE_modelDiet.update({'chooseDiet':'default'})


# BCF 
settings_BCF = {
    'chooseStudyType': 'Martin BCF',
    'chooseDiet': 'zero',
    'chooseEd': 'Martin',
    'chooseRenal': 'off',
    'chooseRenal': 'off'
}


# BMF
settings_BMF = {
    'chooseStudyType': 'Martin BMF',
    'chooseDiet': 'Martin BMF',
    'chooseEd': 'Martin', 
    'chooseRenal': 'off',
    'chooseRenal': 'off'
}



In [ ]:
# Set up input files

BCF_inputFiles = {
    'numSpecies': 2,
    'oceanData': pd.read_csv('data/oceanData_BCFBMF.csv', index_col=0).value,
    'chemicalParams': pd.read_csv('data/chemicalParameters.csv', index_col=0),
    'chemicalData': pd.read_csv('data/chemicalData_BCF.csv',index_col=0),
    'organismData': pd.read_csv('data/organismData_BCF.csv',index_col=0), 
    'foodWebData': pd.read_csv('data/foodWebTable_BCFBMF.csv',index_col=0) 
}


BMF_inputFiles = {
    'numSpecies': 2,
    'oceanData': pd.read_csv('data/oceanData_BCFBMF.csv', index_col=0).value,
    'chemicalParams': pd.read_csv('data/chemicalParameters.csv', index_col=0),
    'chemicalData': pd.read_csv('data/chemicalData_BMF.csv',index_col=0),
    'organismData': pd.read_csv('data/organismData_BMF.csv',index_col=0), 
    'foodWebData': pd.read_csv('data/foodWebTable_BCFBMF.csv',index_col=0) 
}


GE_inputFiles = {
    'organismData': pd.read_csv('data/organismData_Munoz.csv',index_col=0),
    'chemicalData': pd.read_csv('data/chemicalData_Munoz.csv',index_col=0),
    'numSpecies': 12,
    'oceanData': pd.read_csv('data/oceanData_Munoz.csv', index_col=0).Munoz,
    'chemicalParams': pd.read_csv('data/chemicalParameters.csv', index_col=0),
    'foodWebData': pd.read_csv('data/foodWebTable_Munoz.csv',index_col=0), 
    'dietData': pd.read_excel('data/dietData_Munoz.xlsx',sheet_name='median',index_col=0)
}


## 3. BCF + BMF models (+ renal elimination calculations)

In [ ]:
# read in empirical renal elimination data

NgData = pd.DataFrame.from_dict({'PFHxS':0.17, 'PFOS':0.74, 'PFOA':0.48, 'PFNA':0.74, 'PFDA':0.857, 'PFUA':0.95},orient='index',columns=['kElimRatio'])
# Ng & Hungerbuhler 2014, SI Table S7
# b(reab) / b(clear)

Consoer_PFOSratio = 2.01/2.54 # clearance / total elimination ratio, Consoer et al. 2014
Consoer_PFOAratio = 0.12/1.47 # clearance / total elimination ratio, 


### Recalculate BCF with renal elimination

In [ ]:
# run BCF model without renal elimination

# run original model & save key parameters for recalculation 
PFAA_List = ['PFHxS','PFOS','PFOA','PFNA','PFDA','PFUA']
BCF_dict = {}

for PFAA in PFAA_List: 
    results = BioaccumulationModel(PFAA=PFAA, settings_dict=settings_BCF, **BCF_inputFiles).transpose()['Fish']
    results = results[['C_B','C_WTO','k1','k2','kd','ke','Total Elimination']]
    BCF_dict.update({PFAA:results})
    
PFAA_BCFs = pd.DataFrame(BCF_dict).transpose()
PFAA_BCFs.loc['PFNA','C_B'] = np.nan
PFAA_BCFs.loc['PFNA','C_WTO'] = np.nan
PFAA_BCFs['logBCF'] = [np.log10(x) for x in PFAA_BCFs.C_B / (PFAA_BCFs.C_WTO * 1000)]


In [ ]:
# calculate renal elimination rates

PFAA_BCFs['kElimRatios'] = 1 / NgData.kElimRatio
PFAA_BCFs['kr_est'] = np.nan

## Step 1: PFOS & PFOA (algebra starting with kb / (kr + kb) = Consoer ratio); and PFNA = PFOS
PFAA_BCFs.loc['PFOS','kr_est'] = PFAA_BCFs.loc['PFOS','k2'] * ((1- Consoer_PFOSratio) / Consoer_PFOSratio)
PFAA_BCFs.loc['PFOA','kr_est'] = PFAA_BCFs.loc['PFOA','k2'] * ((1- Consoer_PFOAratio) / Consoer_PFOAratio)
PFAA_BCFs.loc['PFNA','kr_est'] = PFAA_BCFs.loc['PFOS','kr_est']


## Step 2: Regression

# set up regression data
df_BCFreg = PFAA_BCFs[['kElimRatios','kr_est']]
df_BCFreg = df_BCFreg.loc[['PFOS','PFOA']]
df_BCFreg.loc['PFDoDA'] = [1, 0]

# extract coefficients from regression 
model_BCF = smf.ols(formula = 'kr_est ~ kElimRatios', data=df_BCFreg).fit()
coeff_BCF = model_BCF.params.kElimRatios
bint_BCF = model_BCF.params.Intercept

# create dummy variables for plotting the linear regression line
df_BCFreg_test = df_BCFreg.copy()
df_BCFreg_test['kElimRatios'] = [0,1,2.5]
pred_BCF = model_BCF.get_prediction(df_BCFreg_test).summary_frame(0.05)


## Step 3: Estimate values for other PFAAs

for PFAA in PFAA_BCFs.index:
    kr_calc = (PFAA_BCFs.loc[PFAA,'kElimRatios'] * coeff_BCF) + bint_BCF # calculate kr from the regression
    if (PFAA_BCFs.isnull().loc[PFAA,'kr_est']):  # use the regression-calculated value if an empirically-based value doesn't exist (i.e. everything but PFOA & PFOS)
        if kr_calc > 0:
            PFAA_BCFs.loc[PFAA,'kr_est'] = kr_calc 
        elif kr_calc < 0:
            PFAA_BCFs.loc[PFAA,'kr_est'] = 0 # if the regression-calculated value is negative, use 0 instead (i.e. PFUA)
            
            
## Step 4: Calculate renal elimination % of totals 
PFAA_BCFs['kr/kr+kb'] = PFAA_BCFs.kr_est / (PFAA_BCFs.kr_est + PFAA_BCFs.k2)
PFAA_BCFs['kr/totelim'] = PFAA_BCFs.kr_est / (PFAA_BCFs.kr_est + PFAA_BCFs.k2 + PFAA_BCFs.ke)
PFAA_BCFs['kr/kb'] = PFAA_BCFs.kr_est / PFAA_BCFs.k2



In [ ]:
# recalculate BCF with renal elimination

PFAA_BCFs['Cb_recalc'] = PFAA_BCFs.C_B * (PFAA_BCFs['Total Elimination']) / (PFAA_BCFs['Total Elimination'] + PFAA_BCFs.kr_est)
PFAA_BCFs['Cb_change'] = PFAA_BCFs.C_B / PFAA_BCFs.Cb_recalc

PFAA_BCFs['BCF_recalc'] = PFAA_BCFs.Cb_recalc / (PFAA_BCFs.C_WTO*1000)
PFAA_BCFs['logBCF_recalc'] = [np.log10(x) for x in PFAA_BCFs.BCF_recalc]


In [ ]:
# add observed data & calculate model bias

PFAA_BCFs['logBCF_obs_WBcalc'] = np.log10([16.8, 1444, 4.3, np.nan, 502, 2876])
PFAA_BCFs['logBCF_obs_carcass'] = np.log10([9.6, 1100, 4, np.nan, 450, 2700])
PFAA_BCFs['BCF_obs_carcass'] = [9.6, 1100, 4, np.nan, 450, 2700]

PFAA_BCFs['MB'] = PFAA_BCFs.BCF_recalc / PFAA_BCFs.BCF_obs_carcass
PFAA_BCFs['MB_original'] = 10**PFAA_BCFs.logBCF / PFAA_BCFs.BCF_obs_carcass
PFAA_BCFs['MBx'] = [x if x > 1 else 1/x for x in PFAA_BCFs.MB]


In [ ]:
# print results

PFAA_BCFs.transpose()

### Recalculate BMF with renal elmination

In [ ]:
# run BMF model without renal elimination

# run original model & save key parameters for recalculation 
PFAS_list = ['PFHxS','PFOS','PFOA','PFNA','PFDA','PFUA']
BMF_dict = {}

for PFAA in PFAA_List: 
    results = BioaccumulationModel(PFAA=PFAA, settings_dict=settings_BMF, **BMF_inputFiles).transpose()['Fish']
    results = results[['C_B','Diet','k1','k2','kd','ke','Total Elimination']]
    BMF_dict.update({PFAA:results})
    
PFAA_BMFs = pd.DataFrame(BMF_dict).transpose()
PFAA_BMFs.loc['PFNA','C_B'] = np.nan
PFAA_BMFs.loc['PFNA','Diet'] = np.nan
PFAA_BMFs['logBMF'] = [np.log10(x) for x in PFAA_BMFs['C_B'] / (PFAA_BMFs['Diet']*1000)]


In [ ]:
# calculate renal elimination rates 

PFAA_BMFs['kElimRatios'] = 1 / NgData.kElimRatio
PFAA_BMFs['kr_est'] = np.nan

## Step 1: PFOS & PFOA (algebra starting with kb / (kr + kb) = Consoer ratio); and PFNA = PFOS
PFAA_BMFs.loc['PFOS','kr_est'] = PFAA_BMFs.loc['PFOS','k2'] * ((1- Consoer_PFOSratio) / Consoer_PFOSratio)
PFAA_BMFs.loc['PFOA','kr_est'] = PFAA_BMFs.loc['PFOA','k2'] * ((1- Consoer_PFOAratio) / Consoer_PFOAratio)
PFAA_BMFs.loc['PFNA','kr_est'] = PFAA_BMFs.loc['PFOS','kr_est']


## Step 2: Regression

# set up regression data
df_BMFreg = PFAA_BMFs[['kElimRatios','kr_est']]
df_BMFreg = df_BMFreg.loc[['PFOS','PFOA']]
df_BMFreg.loc['PFDoDA'] = [1, 0]

# extract coefficients from regression 
model_BMF = smf.ols(formula = 'kr_est ~ kElimRatios', data=df_BMFreg).fit()
coeff_BMF = model_BMF.params.kElimRatios
bint_BMF = model_BMF.params.Intercept

# create dummy variables for plotting the linear regression line
df_BMFreg_test = df_BMFreg.copy()
df_BMFreg_test['kElimRatios'] = [0,1,2.5]
pred_BMF = model_BMF.get_prediction(df_BMFreg_test).summary_frame(0.05)

## Step 3: Estimate values for other PFAAs

for PFAA in PFAA_BMFs.index:
    kr_calc = (PFAA_BMFs.loc[PFAA,'kElimRatios'] * coeff_BMF) + bint_BMF
    if (PFAA_BMFs.isnull().loc[PFAA,'kr_est']): 
        if kr_calc > 0:
            PFAA_BMFs.loc[PFAA,'kr_est'] = kr_calc
        elif kr_calc < 0:
            PFAA_BMFs.loc[PFAA,'kr_est'] = 0

            
## Step 4: Calculate renal elimination % of totals
PFAA_BMFs['kr/kr+kb'] = PFAA_BMFs.kr_est / (PFAA_BMFs.kr_est + PFAA_BMFs.k2)
PFAA_BMFs['kr/totelim'] = PFAA_BMFs.kr_est / (PFAA_BMFs.kr_est + PFAA_BMFs.k2 + PFAA_BMFs.ke)
PFAA_BMFs['kr/kb'] = PFAA_BMFs.kr_est / PFAA_BMFs.k2


In [ ]:
# recalculate BMF with renal elimination

PFAA_BMFs['Cb_recalc'] = PFAA_BMFs.C_B * (PFAA_BMFs['Total Elimination']) / (PFAA_BMFs['Total Elimination'] + PFAA_BMFs.kr_est)
PFAA_BMFs['Cb_change'] = PFAA_BMFs.C_B / PFAA_BMFs.Cb_recalc

PFAA_BMFs['BMF_recalc'] = PFAA_BMFs.Cb_recalc / (PFAA_BMFs.Diet*1000)
PFAA_BMFs['logBMF_recalc'] = [np.log10(x) for x in PFAA_BMFs['BMF_recalc']]
 

In [ ]:
# add observed data & calculate model bias
PFAA_BMFs['BAF_obs'] = [0.14,0.32,0.038,np.nan,0.23,0.28] # reported as BAF, not BMF
PFAA_BMFs['logBAF_obs'] = np.log10([0.14,0.32,0.038,np.nan,0.23,0.28]) # reported as BAF, not BMF

PFAA_BMFs['MB'] = PFAA_BMFs['BMF_recalc'] / PFAA_BMFs['BAF_obs']
PFAA_BMFs['MB_original'] = 10**PFAA_BMFs['logBMF'] / PFAA_BMFs['BAF_obs']
PFAA_BMFs['MBx'] = [x if x > 1 else 1/x for x in PFAA_BMFs.MB]

In [ ]:
# print results

PFAA_BMFs.transpose()

### Create renal elimination parameter object 

In [ ]:
# extract renal:branchial elimination ratio to use in other studies! 

df_krkbRatio = pd.DataFrame(PFAA_BMFs.loc[:,'kr/kb']).rename_axis('PFAA').reset_index()
df_krkbRatio['chemID'] = [6,1,8,9,10,11]
df_krkbRatio.set_index('chemID', inplace=True)

df_krkbRatio

### Plot Results (Figure 2 + Figure S8)

In [ ]:
# create dataframe to plot renal elimination %s

plot_REratios = PFAA_BCFs[['kr/totelim']] # rounded to 3 decimal places BCF and BMF are the same 
plot_REratios.loc[:,'PFCA_CNo_est'] = [np.nan, np.nan, np.nan, 8, 10, 11]
plot_REratios.loc[:,'PFSA_CNo_est'] = [6, np.nan, np.nan, np.nan, np.nan, np.nan]
plot_REratios.loc[:,'PFCA_CNo'] = [np.nan, np.nan, 7, np.nan, np.nan, np.nan]
plot_REratios.loc[:,'PFSA_CNo'] = [np.nan, 8, np.nan, np.nan, np.nan, np.nan]

In [ ]:
# Figure 2: BCF + BMF results
%config InlineBackend.figure_format = 'svg'

## List data subsets and their names that you would like to plot

# Labels
C6S_PL = 'C6 PFSA'
C8S_PL = 'C8 PFSA'
C8C_PL = 'C8 PFCA'
C10C_PL = 'C10 PFCA'
C11C_PL = 'C11 PFCA'

C6S_PR = 'C6 PFSA'
C8S_PR = 'C8 PFSA'
C8C_PR = 'C8 PFCA'
C10C_PR = 'C10 PFCA'
C11C_PR = 'C11 PFCA'

DataframeList = [PFAA_BCFs, PFAA_BMFs, PFAA_BCFs, PFAA_BMFs]
DataframeName = ['BCF','BMF','BCF_original','BMF_original']


# Plot figures
fig, ax = plt.subplots(1,2, figsize=(7,3.5))

## PANEL 1: WITHOUT RENAL ELIMINATION RESULTS 
ax[0].set_title('A) Without renal elimination', fontsize='large', loc='right')

# BCF original     
ax[0].scatter(PFAA_BCFs.loc['PFHxS','logBCF'], PFAA_BCFs.loc['PFHxS','logBCF_obs_carcass'], label=C6S_PL, edgecolors='black', facecolor='blueviolet')
ax[0].scatter(PFAA_BCFs.loc['PFOS','logBCF'], PFAA_BCFs.loc['PFOS','logBCF_obs_carcass'], label=C8S_PL, edgecolors='black', facecolors='cornflowerblue')
ax[0].scatter(PFAA_BCFs.loc['PFOA','logBCF'], PFAA_BCFs.loc['PFOA','logBCF_obs_carcass'], label=C8C_PL, edgecolors='black', facecolors='orange')
ax[0].scatter(PFAA_BCFs.loc['PFDA','logBCF'], PFAA_BCFs.loc['PFDA','logBCF_obs_carcass'], label=C10C_PL,edgecolors='black', facecolors='deeppink') 
ax[0].scatter(PFAA_BCFs.loc['PFUA','logBCF'], PFAA_BCFs.loc['PFUA','logBCF_obs_carcass'], label=C11C_PL, edgecolors='black', facecolors='limegreen') 
        
# BMF original
ax[0].scatter(PFAA_BMFs.loc['PFHxS','logBMF'], PFAA_BMFs.loc['PFHxS','logBAF_obs'], label=C6S_PR, marker='s', edgecolors='black', facecolor='blueviolet')
ax[0].scatter(PFAA_BMFs.loc['PFOS','logBMF'], PFAA_BMFs.loc['PFOS','logBAF_obs'], label=C8S_PR, marker='s', edgecolors='black', facecolors='cornflowerblue')
ax[0].scatter(PFAA_BMFs.loc['PFOA','logBMF'], PFAA_BMFs.loc['PFOA','logBAF_obs'], label=C8C_PR, marker='s',edgecolors='black', facecolors='orange')
ax[0].scatter(PFAA_BMFs.loc['PFDA','logBMF'], PFAA_BMFs.loc['PFDA','logBAF_obs'], label=C10C_PR, marker='s', edgecolors='black', facecolors='deeppink') 
ax[0].scatter(PFAA_BMFs.loc['PFUA','logBMF'], PFAA_BMFs.loc['PFUA','logBAF_obs'], label=C11C_PR, marker='s', edgecolors='black', facecolors='limegreen') 

## PANEL 2: WITH RENAL ELIMINATION RESULTS
ax[1].set_title('B) With renal elimination', fontsize='large', loc='right')

# BCF
ax[1].scatter(PFAA_BCFs.loc['PFHxS','logBCF_recalc'], PFAA_BCFs.loc['PFHxS','logBCF_obs_carcass'], label=C6S_PL, edgecolors='black', facecolors='blueviolet')
ax[1].scatter(PFAA_BCFs.loc['PFOS','logBCF_recalc'], PFAA_BCFs.loc['PFOS','logBCF_obs_carcass'], label=C8S_PL, edgecolors='black', facecolors='cornflowerblue')
ax[1].scatter(PFAA_BCFs.loc['PFOA','logBCF_recalc'], PFAA_BCFs.loc['PFOA','logBCF_obs_carcass'], label=C8C_PL, edgecolors='black', facecolors='orange')
ax[1].scatter(PFAA_BCFs.loc['PFDA','logBCF_recalc'], PFAA_BCFs.loc['PFDA','logBCF_obs_carcass'], label=C10C_PL,edgecolors='black', facecolors='deeppink') 
ax[1].scatter(PFAA_BCFs.loc['PFUA','logBCF_recalc'], PFAA_BCFs.loc['PFUA','logBCF_obs_carcass'], label=C11C_PL,edgecolors='black', facecolors='limegreen') 

# BMF 
ax[1].scatter(PFAA_BMFs.loc['PFHxS','logBMF_recalc'], PFAA_BMFs.loc['PFHxS','logBAF_obs'], label=C6S_PR, marker='s', edgecolors='black',facecolors='blueviolet')
ax[1].scatter(PFAA_BMFs.loc['PFOS','logBMF_recalc'], PFAA_BMFs.loc['PFOS','logBAF_obs'], label=C8S_PR, marker='s', edgecolors='black',facecolors='cornflowerblue')
ax[1].scatter(PFAA_BMFs.loc['PFOA','logBMF_recalc'], PFAA_BMFs.loc['PFOA','logBAF_obs'], label=C8C_PR, marker='s', edgecolors='black',facecolors='orange')
ax[1].scatter(PFAA_BMFs.loc['PFDA','logBMF_recalc'], PFAA_BMFs.loc['PFDA','logBAF_obs'], label=C10C_PR, marker='s', edgecolors='black',facecolors='deeppink') 
ax[1].scatter(PFAA_BMFs.loc['PFUA','logBMF_recalc'], PFAA_BMFs.loc['PFUA','logBAF_obs'], label=C11C_PR, marker='s', edgecolors='black',facecolors='limegreen') 

# legend
ax[1].scatter(-3,-3,label='BCF',edgecolor='black',facecolor='none',lw=2)
ax[1].scatter(-3,-3,label='BMF',marker='s',edgecolor='black',facecolor='none',lw=2)


for i in [0,1]:
    x = np.arange(-3,7)
    y = x
    y1 = x-1
    y2 = x + 1
    y3 = x - np.log10(2)
    y4 = x + np.log10(2) 

    ax[i].plot(x, y, c='black')
    ax[i].plot(x, y1, c='black', linestyle='dotted')
    ax[i].plot(x, y2, c='black', linestyle='dotted')
    ax[i].plot(x, y3, c='black', linestyle='dotted')
    ax[i].plot(x, y4, c='black', linestyle='dotted')

    ax[i].set_ylim(-2,4)
    ax[i].set_xlim(-2,4)    

    ax[i].set_xlabel('Modeled log BCF (L/kg) \n or log BMF (kg/kg)', fontsize='large')
    ax[i].set_ylabel('Observed log BCF (L/kg) \n or log BMF (kg/kg)', fontsize='large')
    
    ax[i].set_facecolor('whitesmoke')
    
fig.tight_layout()

axLine, axLabel = ax[0].get_legend_handles_labels()

fig.legend(axLine[0:5], axLabel[0:5], bbox_to_anchor=(1.14, 0.77),
            borderaxespad=-0.5, frameon=False,fontsize=11)
axLine2, axLabel2 = ax[1].get_legend_handles_labels()

fig.legend(axLine2[10:12], axLabel2[10:12], bbox_to_anchor=(1.07,0.38),
            borderaxespad=-1, fontsize=11, frameon=False, facecolor='inherit')



In [ ]:
# Figure S8: Renal elimination estimation method (linear regression)
%config InlineBackend.figure_format = 'svg'

fig, ax = plt.subplots(1,1, figsize=(3.5,3.5))
plot = ax      

# for legend
plot.scatter(-3,-3, label='C8 PFSA', edgecolor='black',facecolor='cornflowerblue')
plot.scatter(-3,-3, label='C8 PFCA', edgecolor='black',facecolor='orange')
plot.scatter(-3,-3, label='C12 PFCA', edgecolor='black',facecolor='black')
plot.scatter(-3,-3,label='BCF',edgecolor='black',facecolor='none',lw=2)
plot.scatter(-3,-3,label='BMF',marker='s',edgecolor='black',facecolor='none',lw=2)
    
# for BCF
plot.scatter(df_BCFreg.loc['PFOA','kElimRatios'], df_BCFreg.loc['PFOA','kr_est'], edgecolor='black', facecolor='orange')
plot.scatter(df_BCFreg.loc['PFOS','kElimRatios'], df_BCFreg.loc['PFOS','kr_est'], edgecolor='black', facecolor='cornflowerblue')
plot.scatter(df_BCFreg.loc['PFDoDA','kElimRatios'], df_BCFreg.loc['PFDoDA','kr_est'], edgecolor='black', facecolor='black')
plot.plot(df_BCFreg_test.kElimRatios, pred_BCF['mean'],'-',c='black',label='BCF')

# for BMF
plot.scatter(df_BMFreg.loc['PFOA','kElimRatios'], df_BMFreg.loc['PFOA','kr_est'], marker='s', edgecolor='black', facecolor='orange')
plot.scatter(df_BMFreg.loc['PFOS','kElimRatios'], df_BMFreg.loc['PFOS','kr_est'], marker='s', edgecolor='black', facecolor='cornflowerblue')
plot.scatter(df_BMFreg.loc['PFDoDA','kElimRatios'], df_BMFreg.loc['PFDoDA','kr_est'], marker='s', edgecolor='black', facecolor='black')
plot.plot(df_BMFreg_test.kElimRatios, pred_BMF['mean'],'--',c='black',label='BMF')


# for labeling
plot.set_xlim(0.8,2.2)
plot.set_ylim(-0.01,0.10)
plot.set_xlabel('renal clearance-to-reabsorption ratio') # version 2
plot.set_ylabel('est. renal elimination rate ($d^{-1}$)')
    
axLine, axLabel = plot.get_legend_handles_labels()
fig.legend(axLine[2:5], axLabel[2:5], bbox_to_anchor=(1.37,0.77),
            borderaxespad=-0.5, ncol=1, frameon=False,fontsize=11)

axLine2, axLabel2 = plot.get_legend_handles_labels()
fig.legend(axLine[5:7] + axLine[0:2], axLabel[5:7] + axLabel[0:2], bbox_to_anchor=(1.25, 0.52),
          borderaxespad=-0.5, ncol=1, frameon=False, fontsize=11)


## 4. Gironde Estuary model

In [ ]:
# Set up function to run model with and without renal correction**

def runGirondeModel(settings_GE, inputFiles_GE): 
    
    # define model run targets
    
    ParameterList = ['C_B','C_WTO','C_s','k2','ke','kg','Total Elimination']
    PFAA_List = ['PFHxS','PFOS','PFOA','PFNA','PFDA','PFUA']
    noRE_SppList = ['Phy','Pfs','Cop','Mys','Gam','Rag','WSh']

    # run basic model, extract results
    ResultTables = []
    for PFAA in PFAA_List:
        TissueConc = BioaccumulationModel(PFAA=PFAA, settings_dict=settings_GE, **inputFiles_GE)
        temp_table = TissueConc[ParameterList]
        temp_table.insert(0,'PFAA',[PFAA]*len(temp_table)) 

        ResultTables.append(temp_table)

    ResultTable = pd.concat(ResultTables).rename_axis('SppAlias').reset_index()

    ResultTable['log_ngkg'] = np.log10(ResultTable.C_B)
    ResultTable['log_BAF'] = np.log10(ResultTable.C_B / (ResultTable.C_WTO*1000))
    
    # run model again with renal elimination, extract results
    settings_GE.update({'chooseRenal':'on'})
    ResultTables_re = []
    for PFAA in PFAA_List:
        TissueConc = BioaccumulationModel(PFAA=PFAA, settings_dict=settings_GE, **inputFiles_GE)
        temp_table = TissueConc[ParameterList]
        temp_table.insert(0,'PFAA',[PFAA]*len(temp_table)) 

        ResultTables_re.append(temp_table)
        
    ResultTable_re = pd.concat(ResultTables_re).rename_axis('SppAlias').reset_index()
    
    ResultTable_re = ResultTable_re.rename({'C_B':'C_B_re'}, axis='columns')
    ResultTable_re['log_ngkg_re'] = np.log10(ResultTable_re.C_B_re)
    ResultTable_re['log_BAF_re'] = np.log10(ResultTable_re.C_B_re / (ResultTable_re.C_WTO*1000))
    
    # compile results in a single table
    AllResults = pd.merge(ResultTable, ResultTable_re[['SppAlias','PFAA','C_B_re','log_ngkg_re','log_BAF_re']],how = 'outer', on=('SppAlias','PFAA'))
    AllResults['Cb_change'] = AllResults['C_B'] / AllResults['C_B_re']

    # subset the table to the key result values needed for plotting / summary
    AllResults_short = AllResults[['SppAlias','PFAA','log_ngkg','log_ngkg_re','Cb_change']]

    return AllResults, AllResults_short


### Model run

In [ ]:
# 1. Update input files

# prepare diet tables
dietData = pd.read_excel('data/dietData_Munoz.xlsx',sheet_name='median',index_col=0)
dietData_max = pd.read_excel('data/dietData_Munoz.xlsx',sheet_name='max',index_col=0)
dietData_min = pd.read_excel('data/dietData_Munoz.xlsx',sheet_name='min',index_col=0)

# add renal elimination parameterization
GE_inputFiles.update({'kRTable':df_krkbRatio})


In [ ]:
# 2A. Run model w/ and w/o renal elimination - forced Diets

# middle
ModeledMedian_full, ModeledMedian = runGirondeModel(settings_GE_obsDiet, GE_inputFiles)

# max
chemicalData = GE_inputFiles['chemicalData']
chemicalData.rename({'C_WTO':'C_WTO_median','C_WTO_max':'C_WTO'},inplace=True)
chemicalData.rename({'C_s':'C_s_median','C_s_max':'C_s'},inplace=True)
GE_inputFiles.update({'chemicalData':chemicalData})

GE_inputFiles.update({'dietData':dietData_max})

ModeledMaxFULL, ModeledMax = runGirondeModel(settings_GE_obsDiet, GE_inputFiles)
ModeledMax.rename(columns={'log_ngkg':'log_ngkg_max','log_ngkg_re':'log_ngkg_re_max','Cb_change':'Cb_change_max'}, inplace=True)
    

# min
chemicalData = GE_inputFiles['chemicalData']
chemicalData.rename({'C_WTO':'C_WTO_max','C_WTO_min':'C_WTO'},inplace=True)  
chemicalData.rename({'C_s':'C_s_max','C_s_min':'C_s'},inplace=True) 
GE_inputFiles.update({'chemicalData':chemicalData})

GE_inputFiles.update({'dietData':dietData_min})

ModeledMinFULL, ModeledMin = runGirondeModel(settings_GE_obsDiet, GE_inputFiles)
ModeledMin.rename(columns={'log_ngkg':'log_ngkg_min','log_ngkg_re':'log_ngkg_re_min','Cb_change':'Cb_change_min'}, inplace=True)
    
# reset chemicalData
chemicalData = GE_inputFiles['chemicalData']
chemicalData.rename({'C_WTO':'C_WTO_min','C_WTO_median':'C_WTO'},inplace=True)  
chemicalData.rename({'C_s':'C_s_min','C_s_median':'C_s'},inplace=True)
GE_inputFiles.update({'chemicalData':chemicalData})
    
# merge modeled datasets
Modeled_dataframes = [ModeledMedian, ModeledMax, ModeledMin]
Modeled = reduce(lambda left, right: pd.merge(left, right, on=['SppAlias','PFAA'], how='outer'), Modeled_dataframes)
# calculate ranges
Modeled.loc[:,'logngkg_Up'] = Modeled.loc[:,'log_ngkg_max'] - Modeled.loc[:,'log_ngkg']
Modeled.loc[:,'logngkg_Lo'] = Modeled.loc[:,'log_ngkg'] - Modeled.loc[:,'log_ngkg_min']

Modeled.loc[:,'logngkg_re_Up'] = Modeled.loc[:,'log_ngkg_re_max'] - Modeled.loc[:,'log_ngkg_re']
Modeled.loc[:,'logngkg_re_Lo'] = Modeled.loc[:,'log_ngkg_re'] - Modeled.loc[:,'log_ngkg_re_min']

Modeled_v1 = Modeled.copy(deep=True)


In [ ]:
# 2B. Run model w/ and w/o renal elimination - full food web

# middle
GE_inputFiles.update({'dietData':dietData})
ModeledMedian_full, ModeledMedian = runGirondeModel(settings_GE_modelDiet, GE_inputFiles)

# max
chemicalData = GE_inputFiles['chemicalData']
chemicalData.rename({'C_WTO':'C_WTO_median','C_WTO_max':'C_WTO'},inplace=True)
chemicalData.rename({'C_s':'C_s_median','C_s_max':'C_s'},inplace=True)
GE_inputFiles.update({'chemicalData':chemicalData})

GE_inputFiles.update({'dietData':dietData_max})

ModeledMaxFULL, ModeledMax = runGirondeModel(settings_GE_modelDiet, GE_inputFiles)
ModeledMax.rename(columns={'log_ngkg':'log_ngkg_max','log_ngkg_re':'log_ngkg_re_max','Cb_change':'Cb_change_max'}, inplace=True)
    

# min
chemicalData = GE_inputFiles['chemicalData']
chemicalData.rename({'C_WTO':'C_WTO_max','C_WTO_min':'C_WTO'},inplace=True)  
chemicalData.rename({'C_s':'C_s_max','C_s_min':'C_s'},inplace=True)  
GE_inputFiles.update({'chemicalData':chemicalData})

GE_inputFiles.update({'dietData':dietData_min})

ModeledMinFULL, ModeledMin = runGirondeModel(settings_GE_modelDiet, GE_inputFiles)
ModeledMin.rename(columns={'log_ngkg':'log_ngkg_min','log_ngkg_re':'log_ngkg_re_min','Cb_change':'Cb_change_min'}, inplace=True)
    
# reset chemicalData
chemicalData.rename({'C_WTO':'C_WTO_min','C_WTO_median':'C_WTO'},inplace=True)  
chemicalData.rename({'C_s':'C_s_min','C_s_median':'C_s'},inplace=True) 
    
# merge modeled datasets
Modeled_dataframes = [ModeledMedian, ModeledMax, ModeledMin]
Modeled = reduce(lambda left, right: pd.merge(left, right, on=['SppAlias','PFAA'], how='outer'), Modeled_dataframes)
# calculate ranges
Modeled.loc[:,'logngkg_Up'] = Modeled.loc[:,'log_ngkg_max'] - Modeled.loc[:,'log_ngkg']
Modeled.loc[:,'logngkg_Lo'] = Modeled.loc[:,'log_ngkg'] - Modeled.loc[:,'log_ngkg_min']

Modeled.loc[:,'logngkg_re_Up'] = Modeled.loc[:,'log_ngkg_re_max'] - Modeled.loc[:,'log_ngkg_re']
Modeled.loc[:,'logngkg_re_Lo'] = Modeled.loc[:,'log_ngkg_re'] - Modeled.loc[:,'log_ngkg_re_min']

Modeled_v2 = Modeled.copy(deep=True)


In [ ]:
# 3. Read in & format observational data

# add observational data
ObsAvg = dietData.drop('Phy').reset_index().rename(columns={'index':'SppAlias'})
ObsAvg = pd.melt(ObsAvg, id_vars=['SppAlias'],var_name='PFAA',value_name='Obs_ngg' )
ObsAvg['Obs_logngkg'] = np.log10(ObsAvg.Obs_ngg * 1000)

ObsMax = dietData_max.drop('Phy').reset_index().rename(columns={'index':'SppAlias'})
ObsMax = pd.melt(ObsMax, id_vars=['SppAlias'],var_name='PFAA',value_name='Obs_ngg_max' )
ObsMax['Obs_logngkg_max'] = np.log10(ObsMax.Obs_ngg_max * 1000)

ObsMin = dietData_min.drop('Phy').reset_index().rename(columns={'index':'SppAlias'})
ObsMin = pd.melt(ObsMin, id_vars=['SppAlias'],var_name='PFAA',value_name='Obs_ngg_min' )
ObsMin['Obs_logngkg_min'] = np.log10(ObsMin.Obs_ngg_min * 1000)

Obs_dfs = [ObsAvg, ObsMax, ObsMin]
Observed = reduce(lambda left, right: pd.merge(left, right, on=['SppAlias','PFAA'], how='outer'), Obs_dfs)
Observed.loc[:,'Obs_logngkg_Up'] = Observed.loc[:,'Obs_logngkg_max'] - Observed.loc[:,'Obs_logngkg'] # median to the max
Observed.loc[:,'Obs_logngkg_Lo'] = Observed.loc[:,'Obs_logngkg'] - Observed.loc[:,'Obs_logngkg_min'] # min to the median


In [ ]:
# 4. Merge observed with modeled data & manually remove cases which > 50% are NDs

# VERSION 1: forced diets
AllData = pd.merge(Modeled_v1, Observed, how='outer', on=["SppAlias",'PFAA'])

# Remove data for cases in which > 50% are NDs
drop_indx = AllData[(AllData.SppAlias=='Fln') & (AllData.PFAA.isin(['PFOA','PFHxS']))| 
                    (AllData.SppAlias=='Gob') & (AllData.PFAA.isin(['PFOA','PFHxS'])) |
                   (AllData.SppAlias=='CSb') & (AllData.PFAA=='PFHxS') | 
                   (AllData.SppAlias=='Acy')& (AllData.PFAA=='PFOA')|
                   (AllData.SppAlias=='Spr')].index

AllData.iloc[drop_indx,2:len(AllData)] = np.nan 
# instead of deleting these rows, replacing with np.nan allows them to be skipped over in plots
# this just allows the current code for plotting results by individual species to work


# VERSION 2: full food web
AllData_fw = pd.merge(Modeled_v2, Observed, how='outer', on=["SppAlias",'PFAA'])

# Remove data for cases in which > 50% are NDs
drop_indx = AllData_fw[(AllData_fw.SppAlias=='Fln') & (AllData_fw.PFAA.isin(['PFOA','PFHxS']))| 
                    (AllData_fw.SppAlias=='Gob') & (AllData_fw.PFAA.isin(['PFOA','PFHxS'])) |
                   (AllData_fw.SppAlias=='CSb') & (AllData_fw.PFAA=='PFHxS') | 
                   (AllData_fw.SppAlias=='Acy')& (AllData_fw.PFAA=='PFOA')|
                   (AllData_fw.SppAlias=='Spr')].index

AllData_fw.iloc[drop_indx,2:len(AllData_fw)] = np.nan 



### Plot Results (Figures 3 & 4)

In [ ]:
## A. Plot MUNOZ CONCENTRATIONS 1:1 -- 
%config InlineBackend.figure_format = 'svg'

# Select species of interest
OrgType = 'Invertebrates' # Fish or Invertebrates


PlotData = AllData
PlotData.set_index('PFAA',inplace=True, drop=False)

PlotData_fw = AllData_fw
PlotData_fw.set_index('PFAA', inplace=True, drop=False)

# Subset Data
FishData = PlotData[PlotData.SppAlias.isin(['Sol','Fln','Gob','CSb','Acy','Spr'])]
InvData = PlotData[PlotData.SppAlias.isin(['Cop','Mys','Rag','WSh','Gam'])]

FishData_fw = PlotData_fw[PlotData_fw.SppAlias.isin(['Sol','Fln','Gob','CSb','Acy','Spr'])]
InvData_fw = PlotData_fw[PlotData_fw.SppAlias.isin(['Cop','Mys','Rag','WSh','Gam'])]



# List data subsets and their names that you would like to plot
if OrgType == 'Invertebrates':
    DataList = [InvData, InvData_fw]
    PlotTypeList = ['default','default']
    Plot_label = ['A) Inverts, Empirical diet data', 'B) Inverts, Simulated diet data']
elif OrgType == 'Fish':
    DataList = [FishData, FishData_fw]
    PlotTypeList = ['w/ Renal', 'w/ Renal']
    Plot_label = ['A) Fish, Empirical diet data', 'B) Fish, Simulated diet data']

# Set up PFAAs and labels for plot looping
PFAS_List = ['PFHxS','PFOS','PFOA','PFNA','PFDA','PFUA']
PFAS_labels = ['C6 PFSA','C8 PFSA','C8 PFCA', 'C9 PFCA','C10 PFCA','C11 PFCA']
PFAS_colors = ['mediumslateblue','c','orange','violet','deeppink','limegreen']

    
# Plot
fig, ax = plt.subplots(1, 2, figsize=(6.6, 3.5))


for (i, PlotData) in enumerate(DataList): 
    pltfmt = ax[i] if len(DataList) <= 3 else ax[i//cols, i%cols]
    pltfmt.set_title(Plot_label[i], fontsize='large',loc='left') # inverts
    
    PlotType = PlotTypeList[i]
    
    for (j, PFAA) in enumerate(PFAS_List):
        
        if PlotType == 'default':
            pltfmt.errorbar(PlotData.loc[PFAA,'log_ngkg'],PlotData.loc[PFAA,'Obs_logngkg'], label=PFAS_labels[j],
                yerr = [PlotData.loc[PFAA,'Obs_logngkg_Lo'], PlotData.loc[PFAA,'Obs_logngkg_Up']],
                xerr = [PlotData.loc[PFAA,'logngkg_Lo'], PlotData.loc[PFAA,'logngkg_Up']],
                            c=PFAS_colors[j], fmt='o')
        
        elif PlotType == 'w/ Renal':
            pltfmt.errorbar(PlotData.loc[PFAA,'log_ngkg_re'],PlotData.loc[PFAA,'Obs_logngkg'], label=PFAS_labels[j],
                yerr = [PlotData.loc[PFAA,'Obs_logngkg_Lo'], PlotData.loc[PFAA,'Obs_logngkg_Up']],
                xerr = [PlotData.loc[PFAA,'logngkg_re_Lo'], PlotData.loc[PFAA,'logngkg_re_Up']], c=PFAS_colors[j],fmt='o')
            
        
    # 1:1 lines
    x = np.arange (0,7)
    y = x
    y1 = x - 1
    y2 = x + 1
    y3 = x - np.log10(2)
    y4 = x + np.log10(2)

    pltfmt.plot(x, y, c='black')
    pltfmt.plot(x, y1, c='black', linestyle='dotted')
    pltfmt.plot(x, y2, c='black', linestyle='dotted')
    pltfmt.plot(x, y3, c='black', linestyle='dotted') # factor of 2 off
    pltfmt.plot(x, y4, c='black', linestyle='dotted') # factor of 2 off
    pltfmt.set_ylim(0,5.5)
    pltfmt.set_xlim(0,5.5)

# formatting
fig.tight_layout(pad=0.5)
fig.text(0.5, -0.04, 'Modeled log PFAA (ng/kg)', ha='center', fontsize='large')
fig.text(-0.04, 0.5, 'Observed log PFAA (ng/kg)', va='center', rotation='vertical', fontsize='large')
    
pltfmt.legend(loc=1, fontsize='large',frameon=False, bbox_to_anchor=(1.55,0.785))







### Model Metrics

In [ ]:
# 1. Select dataset of interest

Result_type = 'log_ngkg_re' # log_ngkg_re or log_ngkg

# choose species & model type (full 'food web' model or not)
MetricData = pd.concat([InvData]) # FishData, InvData, FishData_fw, etc.

In [ ]:
# 2A. Calculate model bias across species and PFAAs

PFAS_order = ['PFHxS','PFOS','PFOA','PFNA','PFDA','PFUA']
Spp_order = ['Cop','Mys','WSh','Rag','Gam','Acy','Gob','Sol','Fln','CSb']

## Calculate Model Bias by PFAA
MetricData = MetricData.drop(columns=['PFAA'])
MetricData = MetricData[MetricData.Obs_ngg.notnull()] ## drop values with ND observations not shown on plots
MetricData = MetricData.reset_index()

## Calculate Metrics for individual species
MetricData.loc[:,'MB_Tissue'] = np.log10((10**MetricData.loc[:,Result_type]) / (10**MetricData.loc[:,'Obs_logngkg']))
MetricData['AMB_Tissue'] = np.abs(MetricData['MB_Tissue'])
MetricData['MBx_Tissue'] = [10**x if x > 0 else -1/10**x for x in MetricData['MB_Tissue']]
MetricData['AMBx_Tissue'] = 10**MetricData['AMB_Tissue']

MetricData.loc[:,'RMSE_Tissue'] = ((10**MetricData.loc[:,Result_type]) - (10**MetricData.loc[:,'Obs_logngkg']))**2 
MetricData.loc[:,'RMSE_Tissue_log'] = ((MetricData.loc[:,Result_type]) - (MetricData.loc[:,'Obs_logngkg']))**2 

## Display individual species results
#display(MetricData[['PFAA','SppAlias','MB_Tissue','AMB_Tissue','MBx_Tissue','AMBx_Tissue']])



In [ ]:
# 2B. Summarize Model Bias by PFAA 

# first read in MetricData from 2A
ModelMetrics = pd.DataFrame()
ModelMetrics = ModelMetrics.append(MetricData.groupby('PFAA')['MB_Tissue'].mean())
ModelMetrics = ModelMetrics.append(MetricData.groupby('PFAA')['AMB_Tissue'].mean())
ModelMetrics.loc['MBx_Tissue'] = [10**x if x >0 else -1/10**x for x in ModelMetrics.loc['MB_Tissue']]
ModelMetrics.loc['AMBx_Tissue'] = 10**ModelMetrics.loc['AMB_Tissue']

#ModelMetrics = ModelMetrics.append(np.sqrt(MetricData.groupby('PFAA')['RMSE_Tissue'].mean()))
#ModelMetrics = ModelMetrics.transpose().rename_axis('PFAA').reset_index()
ModelMetrics = ModelMetrics.transpose().reindex(PFAS_order)

display(ModelMetrics)


## 5. Additional calculations

### Tissue Type % contribution (Figure 5)

In [ ]:
# Extract partitioning coefficients  
def choose_partitioning(PFAS_list, settings_dict, inputFiles):
    chemicalParams = inputFiles['chemicalParams'][PFAS_list]
    settings = Settings(**settings_dict)
    
    Log_Kow = chemicalParams.loc['Log_Kow_COSMOtherm']
    pKa = chemicalParams.loc['pKa_Armitage'] 
    Log_Dmw = chemicalParams.loc['log_Dmw_droge'] 
    Log_Kpw = np.log10(1.36 * 10**chemicalParams.loc['log_Kpw_Allendorf'])

    # calculate Dow
    def getD_OW (pHi, pKa, K_OW):

        Xn = 1 / (1 + 10**(pHi-pKa)) # neutral fraction of the compound
        Xi = 1 - Xn # ionic fraction of the compound

        logK_OW_i = np.log10(K_OW) - 3.1# based on Armitage et al 2013, see SI Table S7
        K_OW_i = 10 ** logK_OW_i 

        D_OW = Xn * K_OW + Xi * K_OW_i

        if np.log10(D_OW) < 0:
            return "error in D_OW calculation"
        else:
            return D_OW
    
    Log_Dow = []
    pHi = 7.4
    
    for i in range(len(PFAS_list)):
        Dow = getD_OW(pHi, pKa[PFAS_list[i]], 10**Log_Kow[PFAS_list[i]])
        Log_Dow.append(np.log10(Dow))
    Log_Dow = pd.Series(Log_Dow, index=PFAS_list)
    
    
    partitionCoeff = pd.DataFrame({'logKow':Log_Kow, 'logDow':Log_Dow, 'logDmw':Log_Dmw, 'logKpw':Log_Kpw})
    
    return partitionCoeff.transpose()

# Calculate tissue contributions
def calc_TissueContributions(PFAS_list, settings_dict, inputFiles, Spp):
    tissuePct = inputFiles['organismData']
    DataTable = choose_partitioning(PFAS_list, settings_dict, inputFiles)
    
    DataTable.loc['Total (Dbw)'] = (10**DataTable.loc['logDow'] * tissuePct.loc['nu_NB',Spp]) \
        + (10**DataTable.loc['logDmw'] * tissuePct.loc['nu_LB',Spp]) \
        + (10**DataTable.loc['logKpw'] * tissuePct.loc['nu_PB',Spp]) \
        + (10**DataTable.loc['logDow'] * 0.05 * tissuePct.loc['nu_OB',Spp]) + tissuePct.loc['nu_WB',Spp]
    DataTable.loc['Total (log Dbw)'] = np.log10(DataTable.loc['Total (Dbw)'])
    
    DataTable.loc['Lipid%'] = (10**DataTable.loc['logDow'] * tissuePct.loc['nu_NB',Spp]) / DataTable.loc['Total (Dbw)']
    DataTable.loc['Phospholipid%'] = (10**DataTable.loc['logDmw'] * tissuePct.loc['nu_LB',Spp]) / DataTable.loc['Total (Dbw)']
    DataTable.loc['Albumin%'] = (10**DataTable.loc['logKpw'] * tissuePct.loc['nu_PB',Spp]) / DataTable.loc['Total (Dbw)']
    DataTable.loc['NLOM%'] = (10**DataTable.loc['logDow'] * 0.05 * tissuePct.loc['nu_OB',Spp]) / DataTable.loc['Total (Dbw)']
    DataTable.loc['Water%'] = tissuePct.loc['nu_WB',Spp] / DataTable.loc['Total (Dbw)']
    
    return DataTable
    
    

In [ ]:
# STEP 1: Run calculations for all PFAAs and extract logDbws

PFAS_list = ['PFHxS','PFOS','PFOA','PFNA','PFDA','PFUA']

TissueContrib = calc_TissueContributions(PFAS_list, settings_BCF, BCF_inputFiles, 'Fish')

In [ ]:
# STEP 2: Prepare clean table of % results + corresponding Dbw

TC_Table= TissueContrib.loc[['Lipid%','Phospholipid%','Albumin%','NLOM%','Water%'],:]
TC_Table = round(TC_Table*100,2)
TC_Table.loc['log Dbw'] = TissueContrib.loc['Total (log Dbw)']

In [ ]:
# STEP 3: Plot results
%config InlineBackend.figure_format = 'svg'

PFAA_List = TC_Table.columns.tolist()
PFAA_Labels = ['C6 PFSA','C8 PFSA','C8 PFCA','C9 PFCA','C10 PFCA','C11 PFCA']
width = 0.7    

# single plot version
fig,ax=plt.subplots(1,1, figsize=(6,4.5))
plot1 = ax

# Plot 1 - Tissue % Contribution stacked barchart
Lipid = TC_Table.loc['Lipid%']
Phospholipid = TC_Table.loc['Phospholipid%']
Albumin = TC_Table.loc['Albumin%']
NLOM = TC_Table.loc['NLOM%']
Water = TC_Table.loc['Water%']
p1 = plot1.bar(PFAA_List, Phospholipid, width, color='#1f78b4', label='Phospholipid')
p2 = plot1.bar(PFAA_List, Albumin, width, bottom=Phospholipid,color='#e31a1c', label='Plasma Protein')
p3 = plot1.bar(PFAA_List, Lipid, width, bottom=Albumin+Phospholipid, color='#ff7f00', label='Storage Lipid')
p4 = plot1.bar(PFAA_List, NLOM, width, bottom=Lipid+Albumin+Phospholipid, color='#6a3d9a', label='NLOM')
p5 = plot1.bar(PFAA_List, Water, width, bottom=NLOM+Lipid+Albumin+Phospholipid, color='#000000', label='Water')

plot1.set_xticks(np.arange(len(PFAA_List)))
plot1.set_xticklabels(PFAA_Labels,rotation=45, ha='right', size='x-large')
plot1.tick_params(axis='y',labelsize='x-large')
plot1.set_ylabel('Whole-body fish partitioning (%)', size='x-large')
plot1.axvline(x=1.5, c='black',linestyle='--') # add vertical line
plot1.legend(bbox_to_anchor=(1, 0.7),frameon=False, fontsize='large')


plt.tight_layout()

display(round(TC_Table,2))